# Lab session 2- QML- QAIOT (Quantum Variational Classifier)

In [17]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
print("done!")

done!


In [18]:
dev = qml.device("default.qubit", wires=2)

In [19]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])

In [20]:
@qml.qnode(dev, interface="autograd")
def circuit(weights, feature):
    qml.AmplitudeEmbedding(feature,wires=[0,1],pad_with=0.,normalize=True)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

In [21]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [22]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [23]:
def variational_classifier(weights, bias, feature):
    return circuit(weights, feature) + bias


def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)


In [24]:
data = np.loadtxt(r"C:\Users\UTOB\variational_classifer\data\file.txt")
X = data[:, 0:2]
print("First X sample (original)  :", X[0])
# pad the vectors to size 2^2 with constant values

Y = data[:, -1]

First X sample (original)  : [0.4  0.75]


In summary, Following cell randomly shuffles the indices of the data and then splits the features and labels into training and validation sets. The split ratio is 75% for training and 25% for validation. This is a common technique in machine learning to ensure that the model is trained on one subset of the data and validated on another, unseen subset to evaluate its performance.

In [25]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
feats_train = X[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = X[index[num_train:]]
Y_val = Y[index[num_train:]]


In [26]:
num_qubits = 2
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [27]:
opt = NesterovMomentumOptimizer(0.01)
batch_size = 5
# The batch size for mini-batch training is set to 5. This means that during each optimization step, only 5 data points will be used to compute the gradient and update the model parameters.

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(10):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f}  "
        "".format(it + 1, cost(weights, bias, X, Y))
    )

Iter:     1 | Cost: 1.9518638  
Iter:     2 | Cost: 1.8665312  
Iter:     3 | Cost: 1.7408356  
Iter:     4 | Cost: 1.5919188  
Iter:     5 | Cost: 1.4586050  
Iter:     6 | Cost: 1.3433952  
Iter:     7 | Cost: 1.2472508  
Iter:     8 | Cost: 1.1775527  
Iter:     9 | Cost: 1.1098615  
Iter:    10 | Cost: 1.0668143  
